# Run to prepare environment for the challenge

In [1]:
!pip install duckdb
import duckdb

# Create a connection
conn = duckdb.connect(database=':memory:', read_only=False)

# Create the orders table
conn.execute("""
CREATE TABLE orders (
    order_id INTEGER,
    dealership_id INTEGER,
    status VARCHAR,
    total_cost DOUBLE,
    technician VARCHAR,
    order_date DATE
);
""")

# Create the events table
conn.execute("""
CREATE TABLE events (
    event_id INTEGER,
    order_id INTEGER,
    status VARCHAR,
    cost DOUBLE,
    date_time TIMESTAMP
);
""")

# Create the events table
conn.execute("""
CREATE TABLE technicians (
    technician_id INTEGER,
    name VARCHAR,
    expertise VARCHAR
);
""")

# Create the events table
conn.execute("""
CREATE TABLE dealerships (
    dealership_id INTEGER,
    location VARCHAR,
    manager VARCHAR,
    name VARCHAR
);
""")

# Insert some sample data into the orders table
conn.execute("""
INSERT INTO orders VALUES
(100, 2, 'Completed', 85.00, 'James Brown', '2023-08-01'),
(101, 1, 'Completed', 161.0, 'Jane Smith', '2023-08-10'),
(102, 2, 'In Progress', 175.5, 'James Brown', '2023-08-11'),
(103, 3, 'Completed', 216.0, 'Mary Johnson', '2023-08-10'),
(104, 4, 'Completed', 175.0, 'Robert White', '2023-08-11');
""")

# Insert some sample data into the events table
conn.execute("""
INSERT INTO events VALUES
(1, 100, 'In Progress', 0.00, '2023-08-01 08:00:00'),
(1, 100, 'Completed', 85.00, '2023-08-01 09:00:00'),
(1, 101, 'Received', 40.00, '2023-08-10 08:00:00'),
(2, 101, 'In Progress', 50.25, '2023-08-10 10:00:00'),
(3, 102, 'In Progress', 80.00, '2023-08-10 12:00:00'),
(4, 104, 'Received', 40.00, '2023-08-11 08:00:00'),
(5, 104, 'In Progress', 45.00, '2023-08-11 10:00:00'),
(6, 102, 'Received', 40.00, '2023-08-10 08:30:00'),
(7, 102, 'In Progress', 55.50, '2023-08-10 10:30:00'),
(8, 101, 'Completed', 70.75, '2023-08-10 14:00:00'),
(9, 103, 'Received', 60.00, '2023-08-10 09:00:00'),
(10, 103, 'In Progress', 45.50, '2023-08-10 11:00:00'),
(11, 104, 'Completed', 90.00, '2023-08-11 12:30:00'),
(12, 103, 'Completed', 110.50, '2023-08-10 15:00:00');
""")

# Insert sample data into technicians table
conn.execute("""
INSERT INTO technicians VALUES
(1001, 'Jane Smith', 'Brakes'),
(1002, 'James Brown', 'Transmission'),
(1003, 'Mary Johnson', 'Engine'),
(1004, 'Robert White', 'Tires');
""")

# Insert sample data into dealerships table
conn.execute("""
INSERT INTO dealerships VALUES
(1, 'New York', 'John Doe', 'Doe Dealership'),
(2, 'Los Angeles', 'Jane Lee', 'Jane Dealership'),
(3, 'Chicago', 'Jim Bean', 'Jim Bean Ford'),
(4, 'Houston', 'Jill Ray', 'Houston Toyota');
""")


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Analytics Engineer Challenge: Repair Order Analysis

**Background:**

You are an analytics engineer working for a major automotive service provider. Your task is to analyze the repair order data from various dealerships. You have access to several tables in a DuckDB database as prepared in the cells above:

1. `orders`: Contains repair order details including order ID, dealership ID, status, total cost, and technician ID.
2. `events`: Records event updates for each repair order, including changes to status, cost, and timestamps.
3. `technicians`: Information about technicians including their ID, name, and expertise.
4. `dealerships`: Information about each dealership, including its ID, location, and manager.

Your challenge is to write SQL queries to derive meaningful insights from this data, considering the temporal sequence of events and complex relationships between the tables.

**Tasks:**

1. **CTE for Event Summaries**: Create a CTE that summarizes the events for each order, including the earliest and latest status, total cost, and count of events for each order.

2. **Window Function for Running Costs**: Using a window function, calculate the running total of the cost for each dealership, ordered by `date_time` within the events table.

3. **Joining Tables and Analyzing Data**:
    a. Join the `orders`, `events`, `technicians`, and `dealerships` tables.
    b. Provide insights into the progress and cost trends of each order.
    c. Identify the top-performing technicians and dealerships.

4. **Bonus Task (Optional)**: Create a visualization using the results of your analysis that highlights key trends or findings.

**Schema:**

Here are the tables and their respective columns:

- `orders`: `order_id, dealership_id, status, total_cost, technician_id, order_date`
- `events`: `event_id, order_id, status, cost, date_time`
- `technicians`: `technician_id, name, expertise`
- `dealerships`: `dealership_id, location, manager`

**Hints:**

- You may need to use subqueries, aggregation functions, window functions, and CTEs to accomplish these tasks.
- Consider the chronological sequence of events and handle updates or changes to existing orders.
- Explore correlations between the technicians' expertise, the number of orders handled, and other metrics that may lead to interesting insights.

### Deliverables:

Please provide the SQL queries for each of the tasks, along with a brief explanation of the insights or findings derived from the queries. If you choose to complete the bonus task, include the code and visualization as well.

By completing this challenge, you will demonstrate your ability to handle complex SQL queries, utilize advanced features like CTEs and window functions, and derive meaningful insights from a real-world dataset. Feel free to add more data or complexity if it aligns with the goals of your analysis.

Note: The SQL code for creating tables, inserting sample data, and setting up the database environment are executed in this Google Colab notebook in this step. You may need to adjust the data to match the complexity of the challenge.

## Solution 1: CTE for Event Summaries

In [2]:
# Boilerplate for a potential solution
# Define the CTE and query
event_summary_query = """
SELECT * FROM events LIMIT 10;
"""

# Execute the query
result = conn.execute(event_summary_query)

# Fetch the result as a Pandas DataFrame (useful for further analysis or visualization)
df = result.fetchdf()
df

,event_id,order_id,status,cost,date_time
0,1,100,In Progress,0.00,2023-08-01 08:00:00
1,1,100,Completed,85.00,2023-08-01 09:00:00
2,1,101,Received,40.00,2023-08-10 08:00:00
3,2,101,In Progress,50.25,2023-08-10 10:00:00
4,3,102,In Progress,80.00,2023-08-10 12:00:00
5,4,104,Received,40.00,2023-08-11 08:00:00
6,5,104,In Progress,45.00,2023-08-11 10:00:00
7,6,102,Received,40.00,2023-08-10 08:30:00
8,7,102,In Progress,55.50,2023-08-10 10:30:00
9,8,101,Completed,70.75,2023-08-10 14:00:00


## Solution 2: Window Function for Running Costs

In [3]:
running_costs_table = """
SELECT
    e.event_id,
    e.order_id,
    e.cost,
    e.date_time,
    d.dealership_id,
    SUM(e.cost) OVER (PARTITION BY d.dealership_id ORDER BY e.date_time) AS running_total
FROM
    events e
INNER JOIN
    orders o ON e.order_id = o.order_id
INNER JOIN
    dealerships d ON o.dealership_id = d.dealership_id
ORDER BY
    d.dealership_id, e.date_time;
"""

running_costs_result = conn.execute(running_costs_table)

df_running_costs = running_costs_result.fetchdf()
df_running_costs

,event_id,order_id,cost,date_time,dealership_id,running_total
0,1,101,40.00,2023-08-10 08:00:00,1,40.00
1,2,101,50.25,2023-08-10 10:00:00,1,90.25
2,8,101,70.75,2023-08-10 14:00:00,1,161.00
3,1,100,0.00,2023-08-01 08:00:00,2,0.00
4,1,100,85.00,2023-08-01 09:00:00,2,85.00
5,6,102,40.00,2023-08-10 08:30:00,2,125.00
6,7,102,55.50,2023-08-10 10:30:00,2,180.50
7,3,102,80.00,2023-08-10 12:00:00,2,260.50
8,9,103,60.00,2023-08-10 09:00:00,3,60.00
9,10,103,45.50,2023-08-10 11:00:00,3,105.50


# Solution 3: Joining Tables and Analyzing Data:

In [4]:
all_table_join = """
SELECT
    o.order_id,
    o.dealership_id,
    o.status AS order_status,
    o.total_cost AS order_total_cost,
    o.technician AS technician_name,
    o.order_date,
    e.event_id,
    e.status AS event_status,
    e.cost AS event_cost,
    e.date_time,
    t.technician_id,
    t.expertise,
    d.dealership_id,
    d.location AS dealership_location,
    d.manager AS dealership_manager
FROM orders o
LEFT JOIN events e ON o.order_id = e.order_id
LEFT JOIN technicians t ON o.technician = t.name
LEFT JOIN dealerships d ON o.dealership_id = d.dealership_id;

    """

all_table_join_result = conn.execute(all_table_join)

df_all_table_join = all_table_join_result.fetchdf()

df_all_table_join



,order_id,dealership_id,order_status,order_total_cost,technician_name,order_date,event_id,event_status,event_cost,date_time,technician_id,expertise,dealership_id_1,dealership_location,dealership_manager
0,100,2,Completed,85.0,James Brown,2023-08-01,1,In Progress,0.00,2023-08-01 08:00:00,1002,Transmission,2,Los Angeles,Jane Lee
1,100,2,Completed,85.0,James Brown,2023-08-01,1,Completed,85.00,2023-08-01 09:00:00,1002,Transmission,2,Los Angeles,Jane Lee
2,101,1,Completed,161.0,Jane Smith,2023-08-10,1,Received,40.00,2023-08-10 08:00:00,1001,Brakes,1,New York,John Doe
3,101,1,Completed,161.0,Jane Smith,2023-08-10,2,In Progress,50.25,2023-08-10 10:00:00,1001,Brakes,1,New York,John Doe
4,102,2,In Progress,175.5,James Brown,2023-08-11,3,In Progress,80.00,2023-08-10 12:00:00,1002,Transmission,2,Los Angeles,Jane Lee
5,104,4,Completed,175.0,Robert White,2023-08-11,4,Received,40.00,2023-08-11 08:00:00,1004,Tires,4,Houston,Jill Ray
6,104,4,Completed,175.0,Robert White,2023-08-11,5,In Progress,45.00,2023-08-11 10:00:00,1004,Tires,4,Houston,Jill Ray
7,102,2,In Progress,175.5,James Brown,2023-08-11,6,Received,40.00,2023-08-10 08:30:00,1002,Transmission,2,Los Angeles,Jane Lee
8,102,2,In Progress,175.5,James Brown,2023-08-11,7,In Progress,55.50,2023-08-10 10:30:00,1002,Transmission,2,Los Angeles,Jane Lee
9,101,1,Completed,161.0,Jane Smith,2023-08-10,8,Completed,70.75,2023-08-10 14:00:00,1001,Brakes,1,New York,John Doe


Solution for Top Performing Technicians


In [5]:
top_technicians = """
SELECT
    t.technician_id,
    t.name AS technician_name,
    COUNT(o.order_id) AS total_orders_completed
FROM technicians t
LEFT JOIN orders o ON t.name = o.technician  -- Correcting the join condition
GROUP BY t.technician_id, t.name
ORDER BY total_orders_completed DESC;
"""

top_technicians_result = conn.execute(top_technicians)
df_top_technicians = top_technicians_result.fetchdf()
df_top_technicians


,technician_id,technician_name,total_orders_completed
0,1002,James Brown,2
1,1001,Jane Smith,1
2,1003,Mary Johnson,1
3,1004,Robert White,1


Solution for top performing dealerships

In [6]:
top_dealerships = """
SELECT
    d.dealership_id,
    d.location AS dealership_location,
    d.manager AS dealership_manager,
    COUNT(o.order_id) AS total_orders_completed
FROM
    dealerships d
LEFT JOIN
    orders o ON d.dealership_id = o.dealership_id
GROUP BY
    d.dealership_id, d.location, d.manager
ORDER BY
    total_orders_completed DESC;
"""

top_dealerships_result = conn.execute(top_dealerships)

df_top_dealerships = top_dealerships_result.fetchdf()
df_top_dealerships

,dealership_id,dealership_location,dealership_manager,total_orders_completed
0,2,Los Angeles,Jane Lee,2
1,4,Houston,Jill Ray,1
2,1,New York,John Doe,1
3,3,Chicago,Jim Bean,1


## Bonus Task